# ML Project: eCommerce Inventory Prescription

In [25]:
using JuMP
using Gurobi
using CSV
using DataFrames;

In [55]:
# load data
demand = CSV.read("data/salesByWeek.csv", DataFrame)
prices = CSV.read("data/prices.csv", DataFrame)
costs = CSV.read("data/costEstimates.csv", DataFrame) # placeholders
d = Matrix(demand[:,3:end])
r = Vector(prices[:,3]) 
c = Vector(costs[:,2]) # placeholders
tr = Vector(costs[:,3]) # placeholders
fba = 19
leadtime = 8

items_tot = size(d)[1]
time_tot = size(d)[2];

starting_inv = sum(d[:,1:8], dims=2) .+ 10; # starting inv at week 1

#### decision vars ####
x[i,t] --> inventory order from Supplier of product i at time t

j[i,t] --> how much we are selling of product i in time t


#### technically decision vars but not really decisions ####
s[i,t] --> inventory available for sale (at AMZ warehouse) for product i at time t

m[t] --> capital (money) available to purchase inventory at time t

#### params ####
d[i,t] --> demand for product i at time t

r[i] --> sales price for product i

c[i] --> manufacturing cost for product i

tr[i] --> transportation cost for product i


we'll incorporate volume later - for initial, lets assume fixed unit costs

v[i] --> volume (size) of product i

fba --> AMZ storage fee: $19/cbm per month



revenue = 0.70 * sum( d[i,t]*r[i] for i=1:items_tot, t=1:time_tot ) --> amazon takes 30% cut

cost = sum( x[i,t]*(c[i]+tr[i]) for i=1:items_tot, t=1:time_tot ) + sum( fba*s[i,t] for i=1:items_tot, t=1:time_tot:4 )

    --> manu + transport cost of orders + monthly inventory fee for whatever inventory we have


In [56]:
model = Model(with_optimizer(Gurobi.Optimizer))
set_optimizer_attribute(model, "OutputFlag", 0)

@variable(model, x[i=1:items_tot, t=1:time_tot] >= 0) # keeping it at not Int for now
@variable(model, j[i=1:items_tot, t=1:time_tot] >= 0) # keeping it at not Int for now

@variable(model, s[i=1:items_tot, t=1:time_tot] >= 0) # keeping it at not Int for now
@variable(model, m[t=1:time_tot] >= 0)

# how much we sell is bounded by inventory and demand
@constraint(model, [i=1:items_tot, t=1:time_tot], j[i,t] <= s[i,t])
@constraint(model, [i=1:items_tot, t=1:time_tot], j[i,t] <= d[i,t])
# capital is available 2 weeks after a product's sale
@constraint(model, [t=1:(time_tot-2)], m[t+2] == 0.7*sum(j[i,t]*r[i] for i=1:items_tot))
# the cost of inventory orders at time t must be less than our available capital
@constraint(model, [t=1:time_tot], sum(x[i,t]*(c[i]+tr[i]) for i=1:items_tot) <= m[t])
# sales at time t reduce inventory at warehouse, 
# inventory orders at time t are available for sale (arrive at warehouse) 8 weeks later
@constraint(model, [i=1:items_tot, t=leadtime:(time_tot-1)], s[i,t+1] == s[i,t] - j[i,t] + x[i,t-(leadtime-1)])

# address the first 1-8 weeks of supply
@constraint(model, [i=1:items_tot, t=1:leadtime-1], s[i,t+1] == s[i,t] - j[i,t])
@constraint(model, [i=1:items_tot], s[i,1] == starting_inv[i] - j[i,1])

@objective(model, Max, 0.7*sum(j[i,t]*r[i] - x[i,t]*(c[i]+tr[i]) - 0.25*fba*s[i,t] for i=1:items_tot, t=1:time_tot))

optimize!(model)

Academic license - for non-commercial use only - expires 2022-08-19


In [57]:
objective_value(model)

1.4866173800522264e7

In [58]:
value.(x)[:,20:30]

19×11 Matrix{Float64}:
  76.0   84.0   59.0   71.0   65.0   49.0   55.0   59.0   64.0   63.0   72.0
 118.0  113.0  105.0   90.0  112.0   92.0  101.0  102.0   93.0   92.0   82.0
 149.0  125.0  145.0  114.0  106.0  102.0  126.0  111.0  119.0  110.0  101.0
 105.0  109.0  103.0   89.0   93.0   79.0   85.0   91.0   66.0   91.0   96.0
   0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0
   0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0
 280.0  270.0  262.0  234.0  223.0  240.0  222.0  242.0  222.0  240.0  211.0
 212.0  241.0  247.0  223.0  202.0  190.0  192.0  215.0  186.0  200.0  184.0
  91.0   86.0   98.0   86.0   76.0   69.0   75.0   74.0   70.0   66.0   79.0
  87.0   89.0  106.0  105.0   80.0   88.0   68.0   79.0   80.0   83.0   74.0
  99.0  114.0  105.0  122.0  112.0   94.0   94.0   89.0   80.0   99.0  102.0
  15.0   12.0   14.0   14.0   12.0   12.0    6.0   11.0    8.0    9.0    5.0
  13.0   12.0   13.0    6.0   14.0   14.0    8.0    9

In [59]:
value.(s)[:,20:30]

19×11 Matrix{Float64}:
  71.0   74.0   70.0   71.0   64.0   77.0   86.0   81.0   76.0   84.0   59.0
 127.0  139.0  111.0  120.0  109.0  102.0   96.0  103.0  118.0  113.0  105.0
 157.0  167.0  136.0  140.0  137.0  143.0  111.0  142.0  149.0  125.0  145.0
 117.0  113.0  132.0  103.0  105.0   97.0  121.0   99.0  105.0  109.0  103.0
   0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0
   0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0
 328.0  324.0  275.0  266.0  272.0  265.0  285.0  295.0  280.0  270.0  262.0
 288.0  246.0  257.0  212.0  225.0  234.0  241.0  260.0  212.0  241.0  247.0
  84.0   95.0   80.0   90.0   82.0   78.0   94.0   90.0   91.0   86.0   98.0
 105.0  120.0  104.0   86.0   90.0   94.0   83.0   86.0   87.0   89.0  106.0
 144.0  137.0  153.0  113.0  122.0  132.0  126.0  132.0   99.0  114.0  105.0
  17.0   18.0   17.0   12.0    9.0    8.0   12.0    5.0   15.0   12.0   14.0
  11.0   13.0   12.0   13.0   11.0   18.0   12.0   11

In [60]:
value.(j)[:,20:30]

19×11 Matrix{Float64}:
  71.0   74.0   70.0   71.0   64.0   77.0   86.0   81.0   76.0   84.0   59.0
 127.0  139.0  111.0  120.0  109.0  102.0   96.0  103.0  118.0  113.0  105.0
 157.0  167.0  136.0  140.0  137.0  143.0  111.0  142.0  149.0  125.0  145.0
 117.0  113.0  132.0  103.0  105.0   97.0  121.0   99.0  105.0  109.0  103.0
   0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0
   0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0
 328.0  324.0  275.0  266.0  272.0  265.0  285.0  295.0  280.0  270.0  262.0
 288.0  246.0  257.0  212.0  225.0  234.0  241.0  260.0  212.0  241.0  247.0
  84.0   95.0   80.0   90.0   82.0   78.0   94.0   90.0   91.0   86.0   98.0
 105.0  120.0  104.0   86.0   90.0   94.0   83.0   86.0   87.0   89.0  106.0
 144.0  137.0  153.0  113.0  122.0  132.0  126.0  132.0   99.0  114.0  105.0
  17.0   18.0   17.0   12.0    9.0    8.0   12.0    5.0   15.0   12.0   14.0
  11.0   13.0   12.0   13.0   11.0   18.0   12.0   11